In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score
import scipy.stats as stats
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from datetime import datetime
from category_encoders import OrdinalEncoder
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs


import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../Data/training_set_features.csv', index_col='respondent_id')
test = pd.read_csv('../Data/test_set_features.csv', index_col ='respondent_id')
labels = pd.read_csv('../Data/training_set_labels.csv', index_col='respondent_id')

# Imputation

In [3]:
num_cols = train.select_dtypes('number').columns

In [4]:
cat_cols = [
    'race',
    'sex',
    'marital_status',
    'rent_or_own',
    'hhs_geo_region',
    'census_msa',
    'employment_industry',
    'employment_occupation'
]

ord_cols = [
    'age_group',
    'education',
    'income_poverty',
    'employment_status'
]

In [5]:
assert len(num_cols) + len(cat_cols) + len(ord_cols) == train.shape[1]

In [6]:
#Impute Train
for col in num_cols:
    train[col] = train[col].fillna(value=-1)
    test[col] = test[col].fillna(value=-1)

for col in (cat_cols+ord_cols):
    train[col] = train[col].fillna(value='None')
    test[col] = test[col].fillna(value='None')

#### Train Test Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.3, random_state=42)

## Step Forward Selection

In [8]:
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

In [9]:
cat_cols = X_train.select_dtypes('object').columns

In [10]:
ct = ColumnTransformer([('ordinal', OrdinalEncoder(), cat_cols)], remainder='passthrough')

In [11]:
all_cols = X_train.columns

## Feature Selection

In [33]:
sfs2 = SFS(CatBoostClassifier(n_estimators=100, verbose=False),
          k_features = X_train.shape[1],
          forward=True,
          floating=False,
          verbose=2,
          scoring='roc_auc',
          cv=5)

In [50]:
transformed_X_train = ct.fit_transform(X_train)
y_train_h1n1 = np.array(y_train['h1n1_vaccine'])
y_train_seas = np.array(y_train['seasonal_vaccine'])

In [51]:
sfs2 = sfs2.fit(transformed_X_train, y_train_h1n1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   58.5s finished

[2021-07-03 13:30:36] Features: 1/35 -- score: 0.7041935706406306[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:  1.1min finished

[2021-07-03 13:31:40] Features: 2/35 -- score: 0.789321533199932[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:  1.1min finished

[2021-07-03 13:32:46] Features: 3/35 -- score: 0.818183950827421[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1

In [59]:
sfdf = pd.DataFrame(sfs2.get_metric_dict()).T
sfdf

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(21,)","[0.6980246116754346, 0.7073432060062459, 0.700...",0.704194,"(21,)",0.00779196,0.00606241,0.00303121
2,"(21, 27)","[0.7825851991865547, 0.7951155710595109, 0.783...",0.789322,"(21, 27)",0.00687474,0.00534878,0.00267439
3,"(21, 26, 27)","[0.8120544533745861, 0.817046709934313, 0.8154...",0.818184,"(21, 26, 27)",0.00566649,0.00440872,0.00220436
4,"(21, 26, 27, 28)","[0.8380466296825617, 0.8455655357999649, 0.840...",0.844229,"(21, 26, 27, 28)",0.00555931,0.00432533,0.00216266
5,"(10, 21, 26, 27, 28)","[0.8489209554397882, 0.8523308011344524, 0.847...",0.852313,"(10, 21, 26, 27, 28)",0.00454313,0.00353471,0.00176735
6,"(10, 21, 25, 26, 27, 28)","[0.8538523828793987, 0.8561771224453903, 0.851...",0.855243,"(10, 21, 25, 26, 27, 28)",0.00306374,0.0023837,0.00119185
7,"(10, 13, 21, 25, 26, 27, 28)","[0.856787548001646, 0.8566061278188427, 0.8548...",0.857307,"(10, 13, 21, 25, 26, 27, 28)",0.00209319,0.00162857,0.000814285
8,"(10, 13, 21, 22, 25, 26, 27, 28)","[0.8586288561393444, 0.8570530617750662, 0.854...",0.858472,"(10, 13, 21, 22, 25, 26, 27, 28)",0.00342095,0.00266161,0.00133081
9,"(10, 13, 21, 22, 25, 26, 27, 28, 31)","[0.8605365427203978, 0.8597250609144944, 0.858...",0.860156,"(10, 13, 21, 22, 25, 26, 27, 28, 31)",0.00222974,0.00173481,0.000867407
10,"(6, 10, 13, 21, 22, 25, 26, 27, 28, 31)","[0.8635681806927946, 0.8601730620482637, 0.858...",0.861372,"(6, 10, 13, 21, 22, 25, 26, 27, 28, 31)",0.0042222,0.00328502,0.00164251


In [65]:
col_ind_best = sfdf.loc[sfdf['avg_score'] == sfdf['avg_score'].max()].iloc[0]['feature_idx']
col_ind_best

(2, 3, 6, 7, 10, 12, 13, 14, 16, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32)

In [62]:
col_ind_best

19    (2, 3, 6, 7, 10, 12, 13, 14, 16, 21, 22, 25, 2...
Name: feature_idx, dtype: object

# Neural Network

In [12]:
col_ind_best = (2, 3, 6, 7, 10, 12, 13, 14, 16, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32)

In [13]:
best_cols = [all_cols[x] for x in col_ind_best]

In [14]:
best_cols

['behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'doctor_recc_seasonal',
 'child_under_6_months',
 'health_worker',
 'health_insurance',
 'opinion_h1n1_risk',
 'age_group',
 'education',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children']

In [17]:
h1n1_mc = ModelCheckpoint('..Models/h1n1_best_model.h5', monitor='val_auc', mode='max', verbose=1, save_best_only=True)

model_h1n1 = keras.Sequential([
    keras.layers.Dense(46, activation='sigmoid'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [18]:
model_h1n1.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(from_logits=True)])

In [19]:
history = model_h1n1.fit(
    X_train,
    y_train_h1n1,
    batch_size=200,
    epochs=150,
    validation_data=(X_test, y_test_h1n1),
    callbacks=[h1n1_mc]
)

Epoch 1/150
94/94 [==============================] - 1s 4ms/step - loss: 0.5377 - auc: 0.5629 - val_loss: 0.4645 - val_auc: 0.7591

Epoch 00001: val_auc improved from -inf to 0.75906, saving model to ..Models\h1n1_best_model.h5
Epoch 2/150
94/94 [==============================] - 0s 1ms/step - loss: 0.4236 - auc: 0.7944 - val_loss: 0.3993 - val_auc: 0.8121

Epoch 00002: val_auc improved from 0.75906 to 0.81215, saving model to ..Models\h1n1_best_model.h5
Epoch 3/150
94/94 [==============================] - 0s 1ms/step - loss: 0.3810 - auc: 0.8302 - val_loss: 0.3790 - val_auc: 0.8325

Epoch 00003: val_auc improved from 0.81215 to 0.83251, saving model to ..Models\h1n1_best_model.h5
Epoch 4/150
94/94 [==============================] - 0s 1ms/step - loss: 0.3675 - auc: 0.8416 - val_loss: 0.3718 - val_auc: 0.8384

Epoch 00004: val_auc improved from 0.83251 to 0.83840, saving model to ..Models\h1n1_best_model.h5
Epoch 5/150
94/94 [==============================] - 0s 2ms/step - loss: 0.3622

In [20]:
model_h1n1 = load_model('..Models/h1n1_best_model.h5')

y_predicted_h1n1 = model_h1n1.predict(X_test)
roc_auc_score(y_test_h1n1, y_predicted_h1n1)

0.8559017303748202

### Seasonal

In [23]:
from tensorflow import keras
from keras.callbacks import ModelCheckpoint

seas_mc = ModelCheckpoint('..Models/seas_best_model.h5', monitor='val_auc_2', mode='max', verbose=1, save_best_only=True)


model_seas = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_dim=46),
    keras.layers.LeakyReLU(500),
    keras.layers.LeakyReLU(600),
    keras.layers.LeakyReLU(820),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ERROR! Session/line number was not unique in database. History logging moved to new session 194


In [24]:
model_seas.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(from_logits=True)])

In [25]:
history = model_seas.fit(
    X_train,
    y_train_seas,
    batch_size=200,
    epochs=150,
    validation_data=(X_test, y_test_seas),
    callbacks=[seas_mc]
)

Epoch 1/150
94/94 [==============================] - 1s 4ms/step - loss: 0.5647 - auc_2: 0.7867 - val_loss: 0.5117 - val_auc_2: 0.8312

Epoch 00001: val_auc_2 improved from -inf to 0.83121, saving model to ..Models\seas_best_model.h5
Epoch 2/150
94/94 [==============================] - 0s 2ms/step - loss: 0.5015 - auc_2: 0.8342 - val_loss: 0.4913 - val_auc_2: 0.8409

Epoch 00002: val_auc_2 improved from 0.83121 to 0.84087, saving model to ..Models\seas_best_model.h5
Epoch 3/150
94/94 [==============================] - 0s 2ms/step - loss: 0.4925 - auc_2: 0.8403 - val_loss: 0.4853 - val_auc_2: 0.8450

Epoch 00003: val_auc_2 improved from 0.84087 to 0.84502, saving model to ..Models\seas_best_model.h5
Epoch 4/150
94/94 [==============================] - 0s 2ms/step - loss: 0.4912 - auc_2: 0.8415 - val_loss: 0.4832 - val_auc_2: 0.8471

Epoch 00004: val_auc_2 improved from 0.84502 to 0.84713, saving model to ..Models\seas_best_model.h5
Epoch 5/150
94/94 [==============================] - ET

In [26]:
model_seas = load_model('..Models/seas_best_model.h5')

y_predicted_seas = model_seas.predict(X_test)
roc_auc_score(y_test_seas, y_predicted_seas)

0.8571094092016289

In [27]:
y_predicted_h1n1 = model_h1n1.predict(X_test)
y_predicted_seas = model_seas.predict(X_test)
y_predicted = np.concatenate((y_predicted_h1n1, y_predicted_seas), axis=1)
roc_auc_score(y_test, y_predicted)

0.8565055697882246